## AlexNet 
đọc tài liệu  https://www.phamduytung.com/blog/2018-06-15-understanding-alexnet/

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('tmp/data/',one_hot=True)


In [ ]:
learning_rate=0.01
num_step=200
batch_size=128
display=10

# Network parameters

num_input=784
num_class=10
dropout=0.25

# tf Grapd
X=tf.placeholder(tf.float32,[None,num_input])
y=tf.placeholder(tf.float32,[None,num_class])
keep_prob=tf.placeholder(tf.float32)

def conv2d(x,w,b,strider=1):
    x=tf.nn.conv2d(x,w,strides=[1,strider,strider,1],padding='SAME')
    x=tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

def maxpool2d(x,k=2):
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')

# Create model
def conv_net(x,dropout):
    
    weights={
        # filter=11 ,input=3 ,output=96
        'w1':tf.Variable(tf.random_normal([11,11,1,96])),
        'w2':tf.Variable(tf.random_normal([5,5,96,256])),
        'w3':tf.Variable(tf.random_normal([3,3,256,384])),
        'w4':tf.Variable(tf.random_normal([3,3,384,384])),
        'w5':tf.Variable(tf.random_normal([3,3,384,256])),
        'wd1':tf.Variable(tf.random_normal([1*1*256,4096])),
        'wd2':tf.Variable(tf.random_normal([4096,4096])),
        'wd3':tf.Variable(tf.random_normal([4096,num_class]))
        
    }
    biases={
        'b1':tf.Variable(tf.random_normal([96])),
        'b2':tf.Variable(tf.random_normal([256])),
        'b3':tf.Variable(tf.random_normal([384])),
        'b4':tf.Variable(tf.random_normal([384])),
        'b5':tf.Variable(tf.random_normal([256])),
        'bd1':tf.Variable(tf.random_normal([4096])),
        'bd2':tf.Variable(tf.random_normal([4096])),
        'bd3':tf.Variable(tf.random_normal([num_class]))  
    }
    
    x=tf.reshape(x,shape=[-1,28,28,1])
    conv1=conv2d(x,weights['w1'],biases['b1'],strider=4)
    conv1= maxpool2d(conv1,k=3)
    
    conv2=conv2d(conv1,weights['w2'],biases['b2'],strider=1)
    conv2=maxpool2d(conv2,k=3)
    
    conv3=conv2d(conv2,weights['w3'],biases['b3'],strider=1)
    
    conv4=conv2d(conv3,weights['w4'],biases['b4'],strider=1)
    conv4=maxpool2d(conv4,k=1)
    
    conv5=conv2d(conv4,weights['w5'],biases['b5'],strider=1)
    conv5=maxpool2d(conv5,k=1)
    
    # Flatten the multi-dimensional output to feed fully connected layer
    
    feature_map=tf.reshape(conv5,shape=[-1,weights['wd1'].get_shape().as_list()[0]])
    
    # Fully connected layer | Dropout
    
    fc_layer_1=tf.matmul(feature_map,weights['wd1']+biases['bd1'])
    fc_layer_1=tf.nn.dropout(fc_layer_1,dropout)
    
    fc_layer_2=tf.matmul(fc_layer_1,weights['wd2']+biases['bd2'])
    fc_layer_2=tf.nn.dropout(fc_layer_2,dropout)
    
    output=tf.matmul(fc_layer_2,weights['wd3']+biases['bd3'])
    
    return output

logits=conv_net(X,keep_prob)
prediction=tf.nn.softmax(logits)

# Define loss and optimizer

loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

# Evaluate model
correct_pred=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init=tf.global_variables_initializer()


# START TRAINING

with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1,num_step+1):
        batch_x,batch_y=mnist.train.next_batch(batch_size)
        sess.run(optimizer,feed_dict={X:batch_x,y:batch_y,keep_prob:0.8})
        if step%display==0 or step==1:
            loss,acc=sess.run([loss_op,accuracy],feed_dict={
                X:batch_x,
                y:batch_y,
                keep_prob:1.0
            })
        
    print("ok.....")
#     sess.run(accuracy,feed_dict={X:mnist.test.images[:256],
#                                  y:mnist.test.labels[:256],
#                                  keep_prob:1.0})